In [ ]:
import pandas as pd
from functools import reduce
import numpy as np

In [ ]:
# df_dec = pd.read_csv('../ar_reports/orig/kiva_ar_2019_12_31.csv')
# df_dec.drop(columns='Unnamed: 16', inplace=True)

In [ ]:
# df_ar = pd.read_csv('../ar_reports/orig/kiva_ar_2019_12_31.csv')

In [ ]:
# df_ar = pd.read_csv('../ar_reports/orig/legion_of_bloom_detail.csv')

In [ ]:
df_ar = pd.read_csv('../ar_reports/orig/desert_road_ar.csv')

In [ ]:
len(df_ar)

In [ ]:
df_ar.head(1)

separate out the line items that are negative

In [ ]:
df_curr_neg = df_ar[df_ar['current'] < 0]
df_30_neg = df_ar[df_ar['30_days'] < 0]
df_60_neg = df_ar[df_ar['60_days'] < 0]
df_90_neg = df_ar[df_ar['90_days'] < 0]
df_ovr_90_neg = df_ar[df_ar['over_90_days'] < 0]

group together the negative items by ar account

In [ ]:
sum_curr = df_curr_neg.groupby('ar_account', as_index=False)['current'].sum()
sum_30 = df_30_neg.groupby('ar_account', as_index=False)['30_days'].sum()
sum_60 = df_60_neg.groupby('ar_account', as_index=False)['60_days'].sum()
sum_90 = df_90_neg.groupby('ar_account', as_index=False)['90_days'].sum()
sum_ovr_90 = df_ovr_90_neg.groupby('ar_account', as_index=False)['over_90_days'].sum()

In [ ]:
dataframes = [sum_curr, sum_30, sum_60, sum_90, sum_ovr_90]

In [ ]:
df_all_neg = reduce(lambda left, right: pd.merge(left,right, on=['ar_account'], how='outer'), dataframes)

In [ ]:
len(df_all_neg)

In [ ]:
df_all_neg.head(1)

In [ ]:
df_all_neg.fillna(0, inplace=True)

combine all the negatives into one amount

In [ ]:
df_all_neg['amount'] = df_all_neg['current'] + df_all_neg['30_days'] + df_all_neg['60_days'] + df_all_neg['90_days'] + df_all_neg['over_90_days']

separate out the positive line items

In [ ]:
df_curr_pos = df_ar[df_ar['current'] > 0]
df_30_pos = df_ar[df_ar['30_days'] > 0]
df_60_pos = df_ar[df_ar['60_days'] > 0]
df_90_pos = df_ar[df_ar['90_days'] > 0]
df_ovr_90_pos = df_ar[df_ar['over_90_days'] > 0]

In [ ]:
# dataframes_2 = [df_curr_pos, df_30_pos, df_60_pos, df_90_pos, df_ovr_90_pos]

In [ ]:
df_curr_pos.head(1)

take the ar account, days over due, and the amount over

In [ ]:
df_curr_pos.iloc[:2, [2, 8, 11]]

In [ ]:
df_curr_pos_2 = df_curr_pos.iloc[:, [2, 8, 11]]
df_30_pos_2 = df_30_pos.iloc[:, [2, 8 , 12]]
df_60_pos_2 = df_60_pos.iloc[:, [2, 8 , 13]]
df_90_pos_2 = df_90_pos.iloc[:, [2, 8 , 14]]
df_90_ovr_pos_2 = df_ovr_90_pos.iloc[:, [2, 8 , 15]]

for each DPD type, group the positives together

In [ ]:
df_curr_pos_3 = df_curr_pos_2.groupby('ar_account', as_index=False).sum()
df_curr_pos_3.rename(columns={'current': 'amount'}, inplace=True)

In [ ]:
df_30_pos_3 = df_30_pos_2.groupby(['ar_account', 'days_over_due'], as_index=False).sum()
df_30_pos_3.rename(columns={'30_days': 'amount'}, inplace=True)

In [ ]:
df_60_pos_3 = df_60_pos_2.groupby(['ar_account', 'days_over_due'], as_index=False).sum()
df_60_pos_3.rename(columns={'60_days': 'amount'}, inplace=True)

In [ ]:
df_90_pos_3 = df_90_pos_2.groupby(['ar_account', 'days_over_due'], as_index=False).sum()
df_90_pos_3.rename(columns={'90_days': 'amount'}, inplace=True)

In [ ]:
df_90_ovr_pos_3 = df_90_ovr_pos_2.groupby(['ar_account', 'days_over_due'], as_index=False).sum()
df_90_ovr_pos_3.rename(columns={'over_90_days': 'amount'}, inplace=True)

In [ ]:
df_all_pos = df_curr_pos_3.append(df_30_pos_3)

In [ ]:
df_all_pos = df_all_pos.append(df_60_pos_3)

In [ ]:
df_all_pos = df_all_pos.append(df_90_pos_3)

In [ ]:
df_all_pos = df_all_pos.append(df_90_ovr_pos_3)

In [ ]:
df_all_neg.iloc[:2, [0, 6]]

ar_account and amount

In [ ]:
df_all_neg_2 = df_all_neg.iloc[:, [0,6]]

In [ ]:
df_all_neg_2.head(1)

In [ ]:
len(df_all_neg_2)

In [ ]:
len(df_all_pos)

In [ ]:
df_all_pos['in_neg'] = df_all_pos['ar_account'].isin(df_all_neg_2['ar_account']).astype(int)

In [ ]:
df_pos_in = df_all_pos[df_all_pos['in_neg'] == 1]

In [ ]:
df_pos_out = df_all_pos[df_all_pos['in_neg'] == 0]

In [ ]:
df_all_neg_2['in_pos'] = df_all_neg_2['ar_account'].isin(df_pos_in['ar_account']).astype(int)

In [ ]:
df_neg_in = df_all_neg_2[df_all_neg_2['in_pos'] == 1]

In [ ]:
df_neg_out = df_all_neg_2[df_all_neg_2['in_pos'] == 0]

In [ ]:
df_pos_in.reset_index(drop=True, inplace=True)

In [ ]:
df_neg_in.reset_index(drop=True, inplace=True)

In [ ]:
# df_pos_in

In [ ]:
pd.set_option('mode.chained_assignment', None)

for x in range(len(df_neg_in)):
    name_neg = df_neg_in['ar_account'].iloc[x]
    amount_neg = df_neg_in['amount'].iloc[x]
    name_count = df_pos_in['ar_account'].value_counts()[name_neg]
    for i in range(name_count):
        idmax = df_pos_in['days_over_due'].where(df_pos_in['ar_account'] == name_neg).idxmax()
        val1 = df_pos_in['amount'].iloc[idmax] + amount_neg
        if val1 <= 0:
            df_pos_in['amount'].iloc[idmax] = 0
            df_pos_in['days_over_due'].iloc[idmax] = 0
            amount_neg = val1
        else:
            df_pos_in['amount'].iloc[idmax] = val1
            break

In [ ]:
# df_all_pos[df_all_pos['in_neg'] == 1]

In [ ]:
df_all_pos[df_all_pos['days_over_due'] >=270]

In [ ]:
df_over_270 = df_pos_in[df_pos_in['days_over_due'] >= 270]

In [ ]:
df_over_270.head()

In [ ]:
df_groupby = df_over_270.groupby('ar_account', as_index=False)['amount'].sum()

In [ ]:
# df_groupby

In [ ]:
df_delinq = df_groupby[df_groupby['amount'] >= 500]

In [ ]:
# df_delinq

In [ ]:
df_pos_over_270 = df_pos_out[df_pos_out['days_over_due'] >= 270]

In [ ]:
df_pos_groupby = df_pos_over_270.groupby('ar_account', as_index=False)['amount'].sum()

In [ ]:
df_pos_delinq = df_pos_groupby[df_pos_groupby['amount'] >= 500]

In [ ]:
df_all_delinq = df_pos_delinq.append(df_delinq)

In [ ]:
df_all_delinq.to_csv('../ar_reports/orig/no_neg/kiva_delinq_12_31_2019.csv', index=False)

# Test

### Kiva

In [ ]:
# df_ar.drop(columns='Unnamed: 16', inplace=True)

# df_ar.columns = map(str.lower, df_ar.columns)
# df_ar.columns = df_ar.columns.str.replace(" ", "_")

# df_ar.rename(columns={'1-30_days': '30_days', '31-60_days': '60_days', '61-90_days': '90_days', '91+_days': 'over_90_days'}, inplace=True)

# df_ar.rename(columns={'over_90':'over_90_days'}, inplace=True)

# df_ar.replace(" ", 0, inplace=True)

# df_ar[['current', '30_days', '60_days', '90_days', 'over_90']] = df_ar[['current', '30_days', '60_days', '90_days', 'over_90']].apply(pd.to_numeric)

# df_sep[df_sep['state_license'] == 'C9-0000295-LIC']

In [ ]:
# data_neg = [['Rachel', -10], ['Brittany', -5], ['Connor', -15], ['Evelyn', -20]]

In [ ]:
# df_test = pd.DataFrame(data_neg, columns=['Name', 'Amount'])

In [ ]:
# data_pos = [['Rachel', 40, 5], ['Rachel', 30, 5], ['Rachel', 20, 31], ['Brittany', 60, 0], 
#            ['Brittany', 50, 34], ['Brittany', 40, 10], ['Evelyn', 100, 10], ['Evelyn', 90, 5], ['Connor', 0, 42]]

In [ ]:
# df_test_pos = pd.DataFrame(data_pos, columns=['Name', 'Days','Amount'])

In [ ]:
# df_test_pos['Days'].index(max(df_test_pos['Days']))
# df_test_pos['Days'].idxmax()
# df_test_pos['Days'].where(df_test_pos['Name'] == 'Evelyn').idxmax()

In [ ]:
# for x in range(len(df_test)):
#     name_neg = df_test['Name'].iloc[x]
#     amount_neg = df_test['Amount'].iloc[x]
#     name_count = df_test_pos['Name'].value_counts()[name_neg]
#     for i in range(name_count):
#         idmax = df_test_pos['Days'].where(df_test_pos['Name'] == name_neg).idxmax()
#         val1 = df_test_pos['Amount'].iloc[idmax] + amount_neg
#         if val1 <= 0:
#             df_test_pos['Amount'].iloc[idmax] = 0
#             df_test_pos['Days'].iloc[idmax] = 0
#             amount_neg = val1
#         else:
#             df_test_pos['Amount'].iloc[idmax] = val1
#             break

# Old, first run

In [ ]:
# df_30_pos_2.groupby('ar_account', as_index=False).sum()

In [ ]:
# df_curr_pos_2.append(df_30_pos_2, sort=False)

In [ ]:
# df_all_pos = reduce(lambda left, right: pd.merge(left, right, on=['ar_account'], how='left'), dataframes_2)

In [ ]:
# df_sep.columns.get_loc('30_days')

In [ ]:
# df_sep[df_sep['30_days'] > 0].iloc[:,[2, 8, 12]]

In [ ]:
# df_sep_over = df_sep[df_sep['Days Over Due'] >= 270]

In [ ]:
# df_sep_over['91+ Days'] = 
# df_sep_over.astype({'91+ Days': float})
# df_sep_over.info()

In [ ]:
# df_sep_91 = df_sep_over.groupby('AR Account', as_index=False)['91+ Days'].sum()

In [ ]:
# df_sep_bal = df_sep.groupby('AR Account', as_index=False)['Balance Outstanding'].sum()

In [ ]:
# df_sep_merge = pd.merge(df_sep_91, df_sep_bal)

In [ ]:
#doesn't make sense to count because it's including negatives?
# df_sep_over.groupby('AR Account', as_index=False)['91+ Days'].count()

In [ ]:
# df_sep_500 = df_sep_merge[df_sep_merge['91+ Days'] >=500]

In [ ]:
# df_sep_500['percentage'] = df_sep_500['91+ Days'] / df_sep_500['Balance Outstanding']

In [ ]:
# df_sep_500

# Notes

In [ ]:
# df_sep.drop(columns='91+ Days', inplace=True)
# df_sep.astype({'91+ Days':float})
# df_sep.rename(columns={'91+Days': '91+ Days'}, inplace=True)

In [ ]:
# pd.set_option('mode.chained_assignment', None)
# df_sep['current'] = df_sep[['current']].convert_objects(convert_numeric=True).fillna(0)
# df_sep['1-30_days'] = df_sep[['1-30_days']].convert_objects(convert_numeric=True).fillna(0)
# df_sep['31-60_days'] = df_sep[['31-60_days']].convert_objects(convert_numeric=True).fillna(0)
# df_sep['61-90_days'] = df_sep[['61-90_days']].convert_objects(convert_numeric=True).fillna(0)
# df_sep['91+ Days'] = df_sep[['91+ Days']].convert_objects(convert_numeric=True).fillna(0)

# df_sep[['current']].infer_objects().fillna(0)


In [ ]:
# df['A B C D'.split()].apply(np.argsort, axis=1).values
# df_test_pos['Days'].apply(np.argsort).values

In [ ]:
# for x in range(len(df_test)):
#     name_neg = df_test['Name'].iloc[x]
#     amount_neg = df_test['Amount'].iloc[x]
#     name_count = df_test_pos['Name'].value_counts()[name_neg]
#     for i in range(name_count):
#         idmax = df_test_pos['Days'].where(df_test_pos['Name'] == name_neg).idxmax()
#         val1 = df_test_pos['Amount'].iloc[idmax] + amount_neg
#         print(val1)
#         if val1 > 0:
#             print(val1)
#         else:
#             print("nothing")
#             break

In [ ]:
# df_test_pos['Name'].value_counts()
# df_test_pos['Name'].value_counts()['Rachel']

In [ ]:
# a.index(min(a))

In [ ]:
# df_test_pos.index(max(df_test_pos['Days'].where(df_test_pos['Name'] == 'Rachel')))

In [ ]:
# df_test_pos['Days'].get_loc(100)

In [ ]:
# for x in range(len(df_test)):
#     print(df_test['Amount'].iloc[x])
#     val1 = df_test['Amount'].iloc[x]
#     name1 = df_test['Name'].iloc[x]
    
#     df_test_pos['Name'] == name1

# df_test_pos[(df_test_pos['Name'] == 'Rachel') & (df_test_pos['Days'].max())]

# np.argmax(df_test_pos['Days'])

In [ ]:
# df_curr_pos_2.groupby('days_over_due').sum()

In [ ]:
# df_all_neg_2.rename(columns={'total_neg': 'amount'}, inplace=True)

In [ ]:
# df_all_neg_2[df_all_neg_2['ar_account'] == 'Limited Boutique Supply']
# df_all_pos[df_all_pos['ar_account'] == 'Limited Boutique Supply']

In [ ]:

# df_feb = pd.read_csv('../ar_reports/orig/kiva_ar_2020_02_25.csv')

In [ ]:
# pd.to_numeric(df_sep[['current'], ['30_days']])
# df[['A', 'C']].apply(pd.to_numeric) 

In [ ]:
# df_all_pos[df_all_pos['ar_account'] == 'Eaze - NC3 Systems, Inc.']